In [1]:
import numpy as np
from numpy import load

data = load("C:\\Users\\Hp\\Downloads\\QG_jets.npz", allow_pickle = True)
lst = data.files

for item in lst:
    print(item)
    print(data[item])


X
[[[ 2.68769142e-01  3.56903171e-01  4.74138734e+00  2.20000000e+01]
  [ 1.60076377e-01 -2.55609533e-01  4.55022910e+00  2.20000000e+01]
  [ 1.14868731e+00 -6.24380156e-02  4.50385377e+00 -2.11000000e+02]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 1.21266321e+00 -1.12853089e-01  3.04708757e+00 -2.11000000e+02]
  [ 2.40893976e-01 -1.67174886e-02  2.82705667e+00  2.20000000e+01]
  [ 1.02778452e-01 -8.58720522e-02  3.04180579e+00  2.20000000e+01]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 2.16829416e-01 -9.97057017e-01  5.32568913e-01  2.20000000e+01]
  [ 2.31359397e-01 -1.59192211e+00  2.02906587e-01  2.20000000e+01]
  [ 3.41572501e-01 -1.34588077

In [4]:
for k in data.files:
    print(k)

X
y


In [5]:
pip install torch-geometric

     ---------------------------------------- 0.0/661.6 kB ? eta -:--:--
      --------------------------------------- 10.2/661.6 kB ? eta -:--:--
     -- ---------------------------------- 41.0/661.6 kB 393.8 kB/s eta 0:00:02
     --- --------------------------------- 61.4/661.6 kB 465.5 kB/s eta 0:00:02
     ---- -------------------------------- 81.9/661.6 kB 508.4 kB/s eta 0:00:02
     ------ ----------------------------- 112.6/661.6 kB 504.4 kB/s eta 0:00:02
     ------ ----------------------------- 122.9/661.6 kB 450.6 kB/s eta 0:00:02
     ------ ----------------------------- 122.9/661.6 kB 450.6 kB/s eta 0:00:02
     -------- --------------------------- 163.8/661.6 kB 426.7 kB/s eta 0:00:02
     --------- -------------------------- 174.1/661.6 kB 419.0 kB/s eta 0:00:02
     ---------- ------------------------- 194.6/661.6 kB 420.8 kB/s eta 0:00:02
     ----------- ------------------------ 204.8/661.6 kB 401.2 kB/s eta 0:00:02
     ------------ ----------------------- 225.3/661.6

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import networkx as nx
from scipy.sparse import lil_matrix

In [45]:
# Load the dataset
data = np.load("C:\\Users\\Hp\\Downloads\\QG_jets.npz", allow_pickle=True)

In [46]:
# Extract features (X) and labels (y)
X = data["X"]
y = data["y"]

In [51]:
# Define the threshold for connectivity
threshold = 0.5

# Function to calculate distance between particles
def calculate_distance(particle1, particle2):
    delta_pt = particle1[0] - particle2[0]  # Calculate the difference in pt
    delta_rapidity = particle1[1] - particle2[1]  # Calculate the difference in rapidity
    delta_azimuth = particle1[2] - particle2[2]  # Calculate the difference in azimuthal angle
    return np.sqrt(delta_pt**2 + delta_rapidity**2 + delta_azimuth**2)

In [52]:
# Function to create adjacency matrices and edge indices
def create_adjacency_and_edges(X):
    num_samples, num_particles, _ = X.shape
    threshold = 0.1  # Define your distance threshold here

    adjacency_matrices = []
    edge_indices = []

    for sample_idx in range(num_samples):
        G = nx.Graph()
        for i in range(num_particles):
            for j in range(i + 1, num_particles):
                distance = calculate_distance(X[sample_idx, i], X[sample_idx, j])
                if distance < threshold:
                    G.add_edge(i, j)

        adjacency_matrix = nx.adjacency_matrix(G)
        adjacency_matrices.append(adjacency_matrix)
        edge_indices.append(np.array(G.edges()))

    return adjacency_matrices, edge_indices

In [53]:
# Split the dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Create adjacency matrices and edge indices for train, validation, and test sets
adjacency_matrices_train, edge_indices_train = create_adjacency_and_edges(X_train)
adjacency_matrices_val, edge_indices_val = create_adjacency_and_edges(X_val)
adjacency_matrices_test, edge_indices_test = create_adjacency_and_edges(X_test)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14836\1265755108.py:17: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency_matrix = nx.adjacency_matrix(G)


In [ ]:
# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [36]:
# Function to calculate adjacency matrices and edge indices
def calculate_adjacency_matrices(dataset, adjacency_matrices, edge_indices):
    for jet in dataset:
        num_particles = jet.shape[0]

        # Initialize an empty adjacency matrix for the current jet
        adj_matrix = np.zeros((num_particles, num_particles))

        # Calculate pairwise distances and set edges based on the threshold
        for i in range(num_particles):
            for j in range(i + 1, num_particles):
                distance = calculate_distance(jet[i], jet[j])  # Calculate distance between particles
                if np.any(distance < threshold):
                    adj_matrix[i, j] = 1
                    adj_matrix[j, i] = 1

        # Convert the adjacency matrix to edge indices
        edge_index = np.argwhere(adj_matrix == 1).T

        # Append the adjacency matrix and edge indices to the lists
        adjacency_matrices.append(adj_matrix)
        edge_indices.append(torch.tensor(edge_index, dtype=torch.long))

In [ ]:
# Calculate adjacency matrices and edge indices for train, validation, and test sets
calculate_adjacency_matrices(X_train, adjacency_matrices_train, edge_indices_train)
calculate_adjacency_matrices(X_val, adjacency_matrices_val, edge_indices_val)
calculate_adjacency_matrices(X_test, adjacency_matrices_test, edge_indices_test)

In [19]:
# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.fc = nn.Linear(output_dim, 2)  # Binary classification (quark vs. gluon)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch=torch.zeros(x.shape[0], dtype=torch.long))
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [20]:
# Instantiate the GNN model
input_dim = X_train.size(1)
hidden_dim = 64
output_dim = 32
model = GNNModel(input_dim, hidden_dim, output_dim)

In [21]:
# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [22]:
# Training loop
def train(model, optimizer, criterion, loader):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        output = model(data.x, data.edge_index)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


In [23]:
# Validation function
def evaluate(model, criterion, loader):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data in loader:
            output = model(data.x, data.edge_index)
            total_loss += criterion(output, data.y).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(data.y).sum().item()
    loss = total_loss / len(loader)
    accuracy = correct / len(loader.dataset)
    return loss, accuracy

In [24]:
# Create Data objects for train, validation, and test sets
train_data = [Data(x=x, edge_index=edge_index, y=y) for x, edge_index, y in zip(X_train, edge_indices_train, y_train)]
val_data = [Data(x=x, edge_index=edge_index, y=y) for x, edge_index, y in zip(X_val, edge_indices_val, y_val)]
test_data = [Data(x=x, edge_index=edge_index, y=y) for x, edge_index, y in zip(X_test, edge_indices_test, y_test)]


NameError: name 'edge_indices_train' is not defined

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

In [ ]:
# Training loop
num_epochs = 10
train_losses = []
val_losses = []
val_accuracies = []

In [ ]:
for epoch in range(num_epochs):
    train_loss = train(model, optimizer, criterion, train_loader)
    val_loss, val_accuracy = evaluate(model, criterion, val_loader)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.4f}")

In [ ]:
# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss", linestyle="-", marker="o")
plt.plot(val_losses, label="Val Loss", linestyle="-", marker="o")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = evaluate(model, criterion, test_loader)
print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.4f}")

In [39]:
print("Shape of X_train:", X_train.shape)

Shape of X_train: (80000, 139, 4)


In [40]:
print("Example of a single element in X_train:")
print(X_train[0])
print("Shape of the example element:", X_train[0].shape)

Example of a single element in X_train:
[[ 2.38747522e-01  6.83098877e-01  1.86989794e+00  2.20000000e+01]
 [ 4.84468876e-01  6.64770567e-01  1.85677737e+00  2.20000000e+01]
 [ 2.70745579e-01  9.21458554e-01  1.57665564e+00  2.20000000e+01]
 [ 4.10953495e+00  3.59077737e-01  1.72586423e+00  2.11200000e+03]
 [ 1.08357142e+00  3.46838197e-01  1.71266242e+00  2.20000000e+01]
 [ 1.56991976e+00  9.12724948e-01  1.52518512e+00  2.11000000e+02]
 [ 4.30950994e-03  2.90641658e-01  1.47430049e+00  2.20000000e+01]
 [ 5.15456633e+00  3.29756122e-01  1.46554005e+00  2.11000000e+02]
 [ 3.50611414e+00  6.72956010e-01  1.23332919e+00  2.11000000e+02]
 [ 2.69142512e-01  5.81606006e-01  1.74224886e+00  2.20000000e+01]
 [ 1.59410264e+00  3.59610160e-01  1.42731602e+00  2.20000000e+01]
 [ 8.16716389e-02  4.05161741e-01  1.38671542e+00  2.20000000e+01]
 [ 1.15774496e+00  3.83536639e-01  1.46425160e+00 -2.11000000e+02]
 [ 3.46832633e-01  7.95296113e-01  1.47765045e+00  2.20000000e+01]
 [ 2.06130016e+00  6.9